Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Part 3a: Training Workflow using Azure Machine Learning Service

This notebook is showcases Azure Machine Learning Pipelines. To learn more about AML Pipelines, use the following resources:

- https://aka.ms/pl-concept (What are Azure Machine Learning pipelines?)
- https://aka.ms/pl-first-pipeline (Create and run machine learning pipelines)
- https://aka.ms/pl-notebooks (AML Pipelines sample notebooks)

This notebook creates a simple training pipelines which trains the model, evaluates the model, and registers the model.

## Prerequisites
This notebook is designed to be run in Azure ML Notebook VM. See [readme](https://github.com/microsoft/bert-stack-overflow/blob/master/README.md) file for instructions on how to create Notebook VM and open this notebook in it.

### Check Azure Machine Learning Python SDK version

This tutorial requires version 1.0.69 or higher. Let's check the version of the SDK:

In [59]:
import azureml.core

print("Azure Machine Learning Python SDK version:", azureml.core.VERSION)

Azure Machine Learning Python SDK version: 1.0.74


## Stackoverflow Question Tagging Problem 
We will use a simple (dummy) pipeline for now and leave creation of a pipeline using the exact training problem that you used earlier in the workshop as an exercise. Refer to [training notebook](../1-Training/AzureServiceClassifier_Training.ipynb) for details on how you can use an `EstimatorStep` and `PythonScriptStep` to build a train-evaluate-register pipeline. You may also refer to [this repo](https://github.com/microsoft/bert-stack-overflow/), and specifically [this file](https://github.com/microsoft/bert-stack-overflow/blob/master/3-ML-Ops/train-and-register-model.py) for creating a pipeline.

Here, instead of using the script runs as is, we are going to use a AML Pipeline step to do the training.

## Connect to your workspace

In [60]:
from azureml.core import Workspace

workspace = Workspace.from_config()
print('Workspace name: ' + workspace.name, 
      'Azure region: ' + workspace.location, 
      'Subscription id: ' + workspace.subscription_id, 
      'Resource group: ' + workspace.resource_group, sep = '\n')

Workspace name: Auria
Azure region: eastus
Subscription id: 15ae9cb6-95c1-483d-a0e3-b1a1a3b06324
Resource group: PipelinesUsabilityStudy


## Get or create your Compute Target

You have already creared a compute target (`v100cluster` or `p100cluster`) for training. It is time to get that compute cluster.

In [61]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# replace gpu-cluster with your own training cluster name
aml_compute_target = "gpu-cluster"

try:
    aml_compute = AmlCompute(workspace, aml_compute_target)
    print("Found existing compute target.")
except ComputeTargetException:
    print("creating new compute target")
    
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "Standard_NC12s_v3",
                                                                min_nodes = 1, 
                                                                max_nodes = 2)    
    aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
print("Azure Machine Learning Compute attached")

Found existing compute target.
Azure Machine Learning Compute attached


## Get default Datastore

In [62]:
def_blob_store = Datastore(workspace, "workspaceblobstore")
def_blob_store.upload_files(["model/model.pkl"], target_path="model", overwrite=False)
def_blob_store.upload_files(["data/data.csv"], target_path="data", overwrite=False)

print("File uploaded to default datastore")

Uploading an estimated of 1 files
Target already exists. Skipping upload for model\model.pkl
Uploaded 0 files
Uploading an estimated of 1 files
Target already exists. Skipping upload for data\data.csv
Uploaded 0 files
File uploaded to default datastore


## Create Experiment 

In [63]:
from azureml.core import Experiment

experiment_name = 'azml-classifier-using-pipelines' 
experiment = Experiment(workspace, name=experiment_name)
print("Experiment object created.")

Experiment object created.


### Create the Training Step
A Step in a pipeline is a unit of execution. Step typically needs a target of execution (compute target), a script to execute, and may require script arguments and inputs, and can produce outputs. The step also could take a number of other parameters. Azure Machine Learning Pipelines provides the following common built-in Steps (among others). Steps are [documented here](https://docs.microsoft.com/en-us/python/api/azureml-pipeline-steps/azureml.pipeline.steps?view=azure-ml-py).

- PythonScriptStep: Adds a step to run a Python script in a Pipeline.
- DataTransferStep: Transfers data between Azure Blob and Data Lake accounts.
- DatabricksStep: Adds a DataBricks notebook as a step in a Pipeline.
- HyperDriveStep: Creates a Hyper Drive step for Hyper Parameter Tuning in a Pipeline.
- EstimatorStep: Adds a step to run Estimator in a Pipeline.
- AutoMLStep: Creates a AutoML step in a Pipeline.

The following code will create a *PythonScriptStep* to be executed in the Azure Machine Learning Compute we created above using `train.py`, one of the files already made available in the source_directory.

A *PythonScriptStep* is a basic, built-in step to run a Python Script on a compute target. It takes a script name and optionally other parameters like arguments for the script, compute target, inputs and outputs. If no compute target is specified, default compute target for the workspace is used. You can also use a RunConfiguration to specify requirements for the PythonScriptStep, such as conda dependencies and docker image.

> The best practice is to use separate folders for scripts and its dependent files for each step and specify that folder as the source_directory for the step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the source_directory would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the source_directory of the step.

#### Create a runconfig
Need to create a specific [runconfig](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.runconfig.runconfiguration?view=azure-ml-py) for evaluation. Runconfig represents configuration for experiment runs targeting different compute targets in Azure Machine Learning.

### Create Train Step
Create train step.

In [64]:
from azureml.data.data_reference import DataReference
from azureml.pipeline.core import PipelineData
from azureml.pipeline.steps import PythonScriptStep

input_data = DataReference(
    datastore=def_blob_store,
    data_reference_name="train_data",
    path_on_datastore="data/data.csv")

model_dir = PipelineData("model_dir", datastore=def_blob_store)

train_step = PythonScriptStep(
                name="Train Model",
                source_directory='scripts/train',
                script_name="train_model.py", 
                arguments=["--input_data", input_data, "--model_dir", model_dir],
                inputs=[input_data],
                outputs=[model_dir],
                compute_target=aml_compute, 
                allow_reuse=True)
print("train_step created")

train_step created


### Create Evaluate Step
Now, let's create a step to evaluate the model created using the step above.

#### Create the Step using the above runconfig

In [65]:
existing_model = DataReference(
    datastore=def_blob_store,
    data_reference_name="current_model",
    path_on_datastore="model/model.pkl")

eval_result = PipelineData("eval_result", datastore=def_blob_store)

evaluate_step = PythonScriptStep(
                    name="Evaluate Model",
                    source_directory="scripts/evaluate",
                    script_name="evaluate.py",
                    arguments=["--existing_model", existing_model, "--model_dir", model_dir, "--evaluate_result", eval_result],
                    inputs=[existing_model, model_dir],
                    outputs=[eval_result],
                    compute_target=aml_compute, 
                    allow_reuse=True)

print("evaluate_step created")

evaluate_step created


### Create Register Step
Once the model is evaluated, we can register the model.

In [76]:
register_step = PythonScriptStep(
                    name="Register Model",
                    source_directory="scripts/register",
                    script_name="register.py",
                    compute_target=aml_compute,
                    arguments=["--eval_result", eval_result],
                    inputs=[eval_result],
                    allow_reuse=True)
print("register_step created")

register_step created


## Build the pipeline
Once we have the steps (or steps collection), we can build the [pipeline](https://docs.microsoft.com/en-us/python/api/azureml-pipeline-core/azureml.pipeline.core.pipeline.pipeline?view=azure-ml-py). By deafult, all these steps will run in parallel unless there is explicit data dependency (as in the case of this pipeline). 

In [77]:
steps = [train_step, evaluate_step, register_step]

from azureml.pipeline.core import Pipeline

pipeline = Pipeline(workspace=workspace, steps=steps)
print ("Pipeline is built")

Pipeline is built


## Submit a pipeline run 
We can now submit the pipeline using the experiment object.
>If `regenerate_outputs` is set to True, a new submit will always force generation of all step outputs, and disallow data reuse for any step of this run. Once this run is complete, however, subsequent runs may reuse the results of this run.

In [78]:
pipeline_run = experiment.submit(pipeline, regenerate_outputs=False)

Created step Train Model [09bcb236][11737ba5-d804-40b1-aa38-857e13628de1], (This step is eligible to reuse a previous run's output)
Created step Evaluate Model [44868dd3][15f99a93-cb37-41db-9cbd-12d40a479620], (This step is eligible to reuse a previous run's output)
Created step Register Model [a1a86e6f][d8b63d0f-8a7e-45c1-8235-d3e627079c4a], (This step will run and generate new outputs)
Using data reference train_data for StepId [1a6d3b39][af8e5ed1-bd43-41c2-bc6f-414a62821e9a], (Consumers of this data are eligible to reuse prior runs.)
Using data reference current_model for StepId [95a21f23][581be00a-127a-48e0-a41d-410ee93ee0cd], (Consumers of this data are eligible to reuse prior runs.)
Submitted PipelineRun 0f44c2a4-e408-41dc-9477-a31578b8740f
Link to Azure Machine Learning studio: https://ml.azure.com/experiments/azml-classifier-using-pipelines/runs/0f44c2a4-e408-41dc-9477-a31578b8740f?wsid=/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourcegroups/PipelinesUsabilityStudy/w

We can view the current status of the run and stream the logs from within the notebook.

In [81]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [75]:
#pipeline_run.cancel()